# Reference inductor

In [25]:
# donner un pipfile pour la config

from shapeOptInductor import create_plots, update_plots, rot, gen_mesh2, Id  # noqa: F401
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt

## 1 - Geometry and meshing

In [26]:
airgap = 1.74e-3 * np.array([1, 1, 1, 1])
thickness = 1e-2

maxh = 2e-3
mesh = gen_mesh2(airgap, maxh)


XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})

Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## 2 - Computation of magnetic state

In [27]:
# Frequency
f = 5e4  # Hz
omega = 2 * np.pi * f

# Magnetic
mu0 = 4e-7 * np.pi
mur = 1000
mu_iron = mur * mu0
delta = 0.1
mu_coil = np.exp(-1j * delta) * mu0  #  AC losses in the copper from the imaginary part of the permeability

# Current
nb_turn = 200  # Number of turn in the coil
Is = 2  # Source current intensity
js = nb_turn / 2 * Is / (ngs.Integrate(XiCoil, mesh)) * XiCoil  # Source current density


def magWeakFormComplex(a, a_):
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")
    lf = a_ * js * ngs.dx("coil")
    return bf, lf


def solveStateComplex(fes):
    a, a_ = fes.TnT()
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    gf.vec.data = Kinv * F.vec
    return gf, Kinv


fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
a, Kinv = solveStateComplex(fes)
Draw(ngs.Norm(a), mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [28]:
def Inductance(a, mesh):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return  4 * thickness / (2 * Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Losses(a, mesh):
    rel = XiCoil / mu_coil
    return 4 * omega / 2 * thickness * ngs.Integrate(rel.imag * ngs.Norm(ngs.grad(a)) ** 2, mesh)


print(f" L = {Inductance(a,mesh) * 1e6 :.2f} µH")
print(f" P_AC = {Losses(a,mesh) :.2f} W")

 L = 1001.51 µH
 P_AC = 20.35 W


In [29]:
# optim parameters
L_target = 1e-3
l = 0
b = 1


def Constraint(a, mesh):
    return Inductance(a, mesh) - L_target


def dConstraint(a0, p_, mesh=None):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return 2 * 4 * thickness / (2 * Is**2) * rel.real * ngs.InnerProduct(ngs.grad(a0), ngs.grad(p_)) * ngs.dx()


def CostFunction(a, mesh):
    return Losses(a, mesh) + l * Constraint(a, mesh) + 0.5 * b * Constraint(a, mesh) ** 2


def dCostFunction(a0, p_, mesh):
    rel = XiCoil / mu_coil
    dLosses = 2 * 4 * omega / 2 * thickness * rel.imag * ngs.InnerProduct(ngs.grad(a0), ngs.grad(p_)) * ngs.dx()
    return dLosses + l * dConstraint(a0, p_) + b * Constraint(a, mesh) * dConstraint(a0, p_)

# Adjoint

In [30]:
def solveAdjoint(a0, df, Kinv):
    """Solves the adjoint equation for a given df"""
    fes = a0.space
    p, p_ = fes.TnT()
    f = ngs.LinearForm(fes)
    f += -1 * df(a0, p_, fes.mesh)
    f.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = Kinv.H * f.vec
    return gf


p_obj = solveAdjoint(a, dCostFunction, Kinv)
p_const = solveAdjoint(a, dConstraint, Kinv)
Draw(p_const)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 's…

BaseWebGuiScene

## Gradient

In [31]:
from shapeOptInductor import referenceVelocity

v11, v12, v21, v22 = referenceVelocity(mesh)

Draw(v22)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
def SDLosses(a0, p0):
    """Shape derivative for the energy-based losses inside the coil"""
    mesh = a0.space.mesh
    FESReal = ngs.VectorH1(mesh)
    FEScomplex = ngs.VectorH1(mesh, Complex = True)
    X = FEScomplex.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    coeff_losses = 4 * omega / 2 * thickness * (1 / mu_coil).imag
    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans

    dLOmega = ngs.LinearForm(FEScomplex)
    dLOmega += coeff_losses * ngs.InnerProduct(dA * ngs.grad(a0), ngs.grad(a0)) * ngs.dx("coil")
    dLOmega += -js * XiCoil * ngs.div(X) * p0 * ngs.dx
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx
    dLOmega.Assemble()

    dJ = ngs.GridFunction(FESReal)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def shapeGrad(a0, p0, dJ):
    v11, v12, v21, v22 = referenceVelocity(mesh)
    dJlf = dJ(a0, p0)
    FES = ngs.VectorH1(a0.space.mesh)
    dJ = ngs.GridFunction(FES)
    dJ.vec.FV().NumPy()[:] = np.real(dJlf.vec.FV().NumPy()[:])
    g11 = ngs.InnerProduct(dJ, v11)
    g12 = ngs.InnerProduct(dJ, v12)
    g21 = ngs.InnerProduct(dJ, v21)
    g22 = ngs.InnerProduct(dJ, v22)
    return g11, g12, g21, g22


dJ = SDLosses(a, p_obj)


NgException: BaseVector<Complex>::GetIndirect<double> calledin Assemble LinearForm


## Optimization Loop